# 测试控制台

你知道吗？阳见惠凪太可爱了

## POD 构建

In [ ]:
from backend.pipeline import quick_build_pod

res = quick_build_pod(
    nc_path="data/cylinder2d.nc",
    r=128,
    center=True,
    var_keys=("u", "v"),
    verbose=True,   # 打印所有中间过程
    plot=True,      # 画出奇异值谱+累计能量图
)

print("实际使用的模态数 r_used =", res["r_used"])
print("前10阶累计能量:", res["cum_energy"][:10])


## 线性单帧 Demo

In [ ]:
from backend.pipeline import quick_test_linear_baseline

res = quick_test_linear_baseline(
    nc_path="data/cylinder2d.nc",
    pod_dir="artifacts/pod_r128",
    r=128,
    center=True,
    var_keys=("u", "v"),
    frame_idx=64,
    mask_rate=0.02,
    noise_sigma=0.01,
    max_modes=64,      # 前 64 个模态
    modes_per_fig=16,  # 每 16 个为一组：q1-16, q17-32, q33-48, q49-64
    channel=0,
    verbose=True,
)

print("POD 自身截断误差 NMSE =", res["nmse_pod"])
print("线性基线重建 NMSE      =", res["nmse_linear"])


## MLP 单帧 demo

In [ ]:
from backend.pipeline import quick_test_mlp_baseline

res = quick_test_mlp_baseline(
    nc_path="data/cylinder2d.nc",
    pod_dir="artifacts/pod_r128",
    r=128,
    center=True,
    var_keys=("u", "v"),
    frame_idx=64,
    mask_rate=0.02,
    noise_sigma=0.01,
    mlp_noise_sigma=0.01,
    batch_size=64,
    num_epochs=50,
    lr=1e-3,
    max_modes=128,
    modes_per_fig=16,
    channel=0,
    verbose=True,
)

print("NMSE(POD truncation) =", res["nmse_pod"])
print("NMSE(Linear baseline) =", res["nmse_linear"])
print("NMSE(MLP baseline)   =", res["nmse_mlp"])


# 多模态静态展示

In [ ]:
from backend.dataio.io_utils import load_numpy, load_json
from backend.viz.pod_plots import plot_pod_mode_groups
Ur = load_numpy("artifacts/pod/Ur.npy")
r_eff = min(128, Ur.shape[1])
Ur_eff = Ur[:, :r_eff]
meta = load_json("artifacts/pod/pod_meta.json")
H, W, C = meta["H"], meta["W"], meta["C"]
fig_modes = plot_pod_mode_groups(
    Ur_eff,
    H=H,
    W=W,
    C=C,
    max_modes=128,
    group_size=16,
    channel=0,
)
fig_modes.show()

## 完整 sweep + 多尺度分析

In [ ]:
from backend.pipeline import quick_full_experiment

result = quick_full_experiment(
    nc_path="data/cylinder2d.nc",
    var_keys=("u", "v"),
    r=128,
    center=True,
    mask_rates=[0.01, 0.02, 0.05, 0.10],
    noise_sigmas=[0.0, 0.01, 0.02],
    pod_bands={"L": (0, 16), "M": (16, 64), "H": (64, 128)},
    train_mask_rate=0.02,
    train_noise_sigma=0.01,
    max_epochs=50,
    device="cuda",
    verbose=True,
)

# 存盘 + 报告
from backend.eval.reports import save_full_experiment_results, generate_experiment_report_md

paths = save_full_experiment_results(result, "artifacts/experiments", "cylinder_exp1")
report_path = generate_experiment_report_md(result, "artifacts/experiments/cylinder_exp1/report.md",
                                            experiment_name="Cylinder-2D p-σ sweep")

result["df_linear"].head(), result["df_mlp"].head(), report_path


In [ ]:
from backend.pipeline import run_experiment_from_yaml

res = run_experiment_from_yaml(
    "configs/cylinder_exp2.yaml",
    experiment_name="cylinder_exp3_1",
    save_root="artifacts/experiments",
    generate_report=True,
    verbose=True,
)

print("报告路径:", res["report_path"])
res["df_linear"].head()


In [ ]:
from backend.viz.field_plots import plot_example_from_npz

fig = plot_example_from_npz(
    "artifacts/experiments/cylinder_exp2_1/p0-0004_sigma0-01/linear_example_01.npz",
)

In [ ]:
from backend.pipeline import redraw_all_example_figures_from_npz
redraw_all_example_figures_from_npz("artifacts/experiments/cylinder_exp2_1")

In [ ]:
from backend.pipeline import quick_figs_from_saved_experiment

figs = quick_figs_from_saved_experiment("artifacts/experiments", "cylinder_exp1_3")
figs["fig_nmse_vs_mask_linear"]


In [ ]:
from backend.viz.multiscale_plots import save_multiscale_summary_figures_from_dir

paths = save_multiscale_summary_figures_from_dir(
    "artifacts/experiments/cylinder_exp2_1"
)
paths

# 自检模块

In [ ]:
import numpy as np
from backend.fourier.filters import make_wavenumber_grid, fft2_field, ifft2_field, radial_bin_spectrum
from backend.metrics.fourier_metrics import fourier_radial_nrmse_curve, kstar_from_radial_curve, fourier_band_nrmse, energy_spectrum

H, W = 80, 640
x = np.random.randn(H, W)
x_hat = x + 0.1*np.random.randn(H, W)

k, E, edges = energy_spectrum(x, num_bins=64, grid_meta={"dx":1.0, "dy":1.0})
kc, Et, Ee, nrmse = fourier_radial_nrmse_curve(x_hat, x, num_bins=64, grid_meta={"dx":1.0, "dy":1.0})
kstar = kstar_from_radial_curve(kc, nrmse, threshold=1.0)

bands = fourier_band_nrmse(x_hat, x, k_edges=[0.0, kc[len(kc)//3], kc[2*len(kc)//3], kc[-1]], grid_meta={"dx":1.0, "dy":1.0})
print("kstar:", kstar, "bands:", bands)
